# Course Schedule

The purpose of this notebook is prepare a markdown document containing a detailed course schedule.

## Compute University Class Schedules

The first cell computes a list ``classday_dates`` containing date objects for every valid class day in a semester, and a derived dictionary ``class_dates`` with date objects for standard weekly class schedules such as MWF and TR. The class dates account for university holidays and breaks. To update for a new semester, consult the University calendar and then update the first and last class day, the list of University holidays and breaks, and any special rules that may apply to a particular semeester.

In [9]:
from datetime import date, timedelta         

def date_range(start_date, end_date):
    return [start_date + timedelta(days=k) for k in range((end_date - start_date).days + 1)]

def remove_dates(a, b):
    return sorted(set(a) - set(b))

# first and last class days for the semester
first_class_day = date(2022, 1, 10)
last_class_day = date(2022, 4, 26)

# compute all dates in the semester
semester_dates = date_range(first_class_day, last_class_day)

# remove weekends from classdays
weekend_dates = [day for day in semester_dates if day.weekday() in (5, 6)]
classday_dates = remove_dates(semester_dates, weekend_dates)

# university holidays
holidays = {
    'Martin Luther_King': date_range(date(2022, 1, 17), date(2022, 1, 17)),
    'Spring Break': date_range(date(2022, 3, 5), date(2022, 3, 13)),
    'Easter Break': date_range(date(2022, 4, 15), date(2022, 4, 18)),
}

# remove university holidays
for holiday, holiday_dates in holidays.items():
    classday_dates = remove_dates(classday_dates, holiday_dates)
    
# compute standard university class schedules
class_dates = dict()
class_dates["MWF"] = [day for day in classday_dates if day.weekday() in (0, 2, 4)]
class_dates["TR"] = [day for day in classday_dates if day.weekday() in (1, 3)]
class_dates["MW"] = [day for day in classday_dates if day.weekday() in (0, 2)]
class_dates["W"] = [day for day in classday_dates if day.weekday() == 2]
class_dates["F"] = [day for day in classday_dates if day.weekday() == 4]

# special rules for Spring, 2022
class_dates["MWF"].append(class_dates["TR"][-1])
class_dates["TR"] = class_dates["TR"][:-1]

for sched, dates in class_dates.items():
    print(f"{sched:>5s}:  {len(dates):2d} class days")                                        

  MWF:  41 class days
   TR:  28 class days
   MW:  27 class days
    W:  14 class days
    F:  13 class days


## CBE 30338/32338 Labs and Lectures

The following cell adds course specific details. In this instance, we we first modify the ``class_dates`` dictionary to remove any dates for which this class may not meet. Considerations may include lab or tutorial sessions on days prior to Spring or Fall break, or prior to Junior Parents Weekend.

Individual lists are created for each type of course activity, such as lectures, labs, or office hours. The times in each list contain information specific to session. The information has three components: title, pre-reads, and assignment to be distributed on that date.

In [10]:
# enter informaation as (title, pre-readings, assignment)

no_class_days = [
    date(2022, 3, 4),     # no lab on the friday before spring break
    date(2022, 1, 21),    # no lab day
    date(2022, 4, 22),    # no lab day after last class session
]

for sched, dates in class_dates.items():
    class_dates[sched] = remove_dates(class_dates[sched], no_class_days)
    print(sched, len(dates))

labs = [
    ("Lab 1: Coding a Simple Thermostat", "Lab Chapter 1", "Lab Assignment 1"),
#    ("Lab 2: Empirical Model Fitting", "Lab Chapter 2", "Lab Assignment 2"),
    ("Lab 3: Model Parameter Estimation", "Lab Chapter 3", "Lab Assignment 3"),
    ("", "", ""),
    ("Quiz 1 / Lab 4: PID Control", "Lab Chapter 4", "Lab Assignment 4"),
    ("Quiz 2: Feedback Control", "", ""),
    ("Lab 5: Observers and State Estimation", "Lab Chapter 5", "Lab Assignment 5"),
    ("Quiz 3: Process Analytics", "", ""),
    ("Lab 6: Optimal Control", "Lab Chapter 6", "Lab Assignment 6"),
    ("Lab 7: Model Predictive Control", "Lab Chapter 7", "Lab Assignment 7"),
    ("Quiz 4: Optimization", "", ""),
]

lectures = [
    # Course Introduction
    ("Course organization and Introduction to Process Control", "", ""),
    ("Introduction to the Temperature Control Laboratory", "", ""),
    # Modeling
    ("First Linear Systems ", "Section 2.1", "HW Assignment 1"),
    ("Modeling: Pharamakokinetics", "Section 2.2", ""),
    ("Modeling: Temperature Control Lab", "Section 2.3", ""),
    ("Modeling: Second Order Systems", "Section 2.4", ""),
    ("Modeling: Fitting Higher Order Models", "Section 2.5", ""),
    ("Modeling: Reactor Modeling", "Sections 2.6 and 2.7", "HW Assignment 2"),
    # Feedback Control
    ("Setpoints, Disturbances, Servo and Regulation Problems", "", ""),
    ("P, PI, and PID Control, and Tuning", "", ""),
    ("Implementation Issues: Bumpless Transfer and Integral Windup", "", "No Assignment (JPW)"),
    ("Cascade Control", "", ""),
    # Process Analytics
    ("Historians, Open Loop Estimation", "", ""),
    ("Observers and State Estimation", "", ""),
    ("Anomoly Detection", "", "Final Project Assigned"),
    ("State Estimation: Environmental and Medical Applications", "", ""),
    # Optimization
    ("Linear Production Models", "", "Project Proposal Due"),
    ("Linear Blending Problems", "", ""),
    ("Static Operability", "", ""),
    ("Dynamic Optimization", "", ""),
    # Predictive Control
    ("Simulation and Open-Loop Optimal Control", "", ""),
    ("Predictive Control", "", ""),
    ("Implementing Predictive Control", "", "Project Progress Report Due"),
    ("Predictive Control Case Study", "", ""),
    # Discrete Event Modeling
    ("Agent Based Models", "", ""),
    ("Queuing and Batch Chemcial Processes", "", ""),
    # Student Projects
    ("Student Project Presentations", "", ""),
    ("Student Project Presentations", "", ""),
]

office_hours = [
    ("", "", ""),
    ("", "", ""),
    ("", "", ""),
    ("", "", ""),
    ("", "", ""),
    ("", "", ""),
    ("", "", ""),
    ("", "", ""),
    ("Project Reviews", "", ""),
    ("Project Reviews", "", ""),
    ("Project Reviews", "", ""),
    ("", "", ""),
    ("", "", ""),
    ("", "", ""),
]


MWF 41
TR 28
MW 27
W 14
F 13


## Display Course Schedule

In [11]:
import pandas as pd
from datetime import timedelta
from IPython.display import display, HTML, Markdown

# create a class schedule dictionary
def schedule(activity, days, topics=[("", "", "")]*len(class_dates)):
    if len(topics) < len(days):
        topics = topics + [("", "", "")]*(len(days) - len(topics))
    classes = {}
    for k, day in enumerate(sorted(days)):
        classes[day] = {
            "Activity": activity,
            "Topic": topics[k][0],
            "Pre-reading": topics[k][1],
            "Assignment": topics[k][2],
            "k": k + 1,
            "week": day.isocalendar()[1],
        }
    return classes

class_schedule = schedule("Class", class_dates["TR"], lectures)
class_schedule.update(schedule("Lab", class_dates["F"], labs))
class_schedule.update(schedule("Office Hours", class_dates["W"], office_hours))

# create pandas dataframe
class_meetings = pd.DataFrame.from_dict(class_schedule, orient="index")
class_meetings.sort_index(inplace=True)
class_meetings["date"] = class_meetings.index
class_meetings["Day"] = pd.to_datetime(class_meetings.index).strftime("%a")
class_meetings.index = pd.to_datetime(class_meetings.index).strftime("%b %-d")

weeks = class_meetings.groupby("week")

txt = f"# CBE 30338/32338 Schedule\n\n"
for k, week in enumerate(weeks):
    d1 = week[1]["date"].min().strftime("%b %-d")
    d2 = week[1]["date"].max().strftime("%b %-d")
    hdr = f"## Week {k + 1}: {d1} - {d2}"
    txt += hdr + "\n" * 2
    tbl = week[1][["Day", "Activity", "Topic", "Pre-reading", "Assignment"]].to_markdown()
    txt += tbl+ "\n" * 2
    display(Markdown(hdr))
    display(Markdown(tbl))
    
with open("Schedule.md", "w") as f:
    f.write(txt)

## Week 1: Jan 11 - Jan 14

|        | Day   | Activity     | Topic                                                   | Pre-reading   | Assignment       |
|:-------|:------|:-------------|:--------------------------------------------------------|:--------------|:-----------------|
| Jan 11 | Tue   | Class        | Course organization and Introduction to Process Control |               |                  |
| Jan 12 | Wed   | Office Hours |                                                         |               |                  |
| Jan 13 | Thu   | Class        | Introduction to the Temperature Control Laboratory      |               |                  |
| Jan 14 | Fri   | Lab          | Lab 1: Coding a Simple Thermostat                       | Lab Chapter 1 | Lab Assignment 1 |

## Week 2: Jan 18 - Jan 20

|        | Day   | Activity     | Topic                       | Pre-reading   | Assignment      |
|:-------|:------|:-------------|:----------------------------|:--------------|:----------------|
| Jan 18 | Tue   | Class        | First Linear Systems        | Section 2.1   | HW Assignment 1 |
| Jan 19 | Wed   | Office Hours |                             |               |                 |
| Jan 20 | Thu   | Class        | Modeling: Pharamakokinetics | Section 2.2   |                 |

## Week 3: Jan 25 - Jan 28

...

|       | Day   | Activity     | Topic                           | Pre-reading   | Assignment                  |
|:------|:------|:-------------|:--------------------------------|:--------------|:----------------------------|
| Apr 5 | Tue   | Class        | Implementing Predictive Control |               | Project Progress Report Due |
| Apr 6 | Wed   | Office Hours |                                 |               |                             |
| Apr 7 | Thu   | Class        | Predictive Control Case Study   |               |                             |
| Apr 8 | Fri   | Lab          | Quiz 4: Optimization            |               |                             |

## Week 13: Apr 12 - Apr 14

|        | Day   | Activity     | Topic                                | Pre-reading   | Assignment   |
|:-------|:------|:-------------|:-------------------------------------|:--------------|:-------------|
| Apr 12 | Tue   | Class        | Agent Based Models                   |               |              |
| Apr 13 | Wed   | Office Hours |                                      |               |              |
| Apr 14 | Thu   | Class        | Queuing and Batch Chemcial Processes |               |              |

## Week 14: Apr 19 - Apr 21

|        | Day   | Activity     | Topic                         | Pre-reading   | Assignment   |
|:-------|:------|:-------------|:------------------------------|:--------------|:-------------|
| Apr 19 | Tue   | Class        | Student Project Presentations |               |              |
| Apr 20 | Wed   | Office Hours |                               |               |              |
| Apr 21 | Thu   | Class        | Student Project Presentations |               |              |